This file can be used to translate text using Google Cloud. 
In order to run this program you will need a Google Cloud API key. 
Information about the Google Cloud API can be found here: https://console.cloud.google.com/marketplace/product/google/translate.googleapis.com

In [ ]:
# !pip install google.cloud.translate

In [102]:
import os, csv
# public
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import pandas as pd
from tqdm import tqdm
from google.cloud import translate_v2 as translate
import pandas as pd
import ast

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# a method that calls the cloud translator
def translate_text(source: str, target: str, text: str) -> dict:
    """
    Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(
        text
        , source_language=source
        , target_language=target
    )
    return result

def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [292]:
# source language
src_lan = 'en'

# target language
tgt_lan = 'fa'
language = "Farsi"


RESOURCE = 'Sense-Projection'
DATA =  RESOURCE+'/data/'+language
GOLD_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TRANS_FILE = os.path.join(DATA, 'trans-sentences-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'/data/English/Original'

RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'gold-sentences-English.tsv')


In [ ]:
KEY = os.path.join(RESOURCE, 'key')
key_file = "" # replace this with the name of the .json file that holds your google cloud translation API information 
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, key_file)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [256]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
sents = sentence_df["Sentence"].tolist()

In [218]:
sids = sentence_df["ID"].tolist()
sids[:4]

['d000.s000', 'd000.s001', 'd000.s002', 'd000.s003']

# TRANSLATE

In [220]:
# get languages
client = translate.Client()
languages = client.get_languages()
languages

[{'language': 'ab', 'name': 'Abkhaz'},
 {'language': 'ace', 'name': 'Acehnese'},
 {'language': 'ach', 'name': 'Acholi'},
 {'language': 'af', 'name': 'Afrikaans'},
 {'language': 'sq', 'name': 'Albanian'},
 {'language': 'alz', 'name': 'Alur'},
 {'language': 'am', 'name': 'Amharic'},
 {'language': 'ar', 'name': 'Arabic'},
 {'language': 'hy', 'name': 'Armenian'},
 {'language': 'as', 'name': 'Assamese'},
 {'language': 'awa', 'name': 'Awadhi'},
 {'language': 'ay', 'name': 'Aymara'},
 {'language': 'az', 'name': 'Azerbaijani'},
 {'language': 'ban', 'name': 'Balinese'},
 {'language': 'bm', 'name': 'Bambara'},
 {'language': 'ba', 'name': 'Bashkir'},
 {'language': 'eu', 'name': 'Basque'},
 {'language': 'btx', 'name': 'Batak Karo'},
 {'language': 'bts', 'name': 'Batak Simalungun'},
 {'language': 'bbc', 'name': 'Batak Toba'},
 {'language': 'be', 'name': 'Belarusian'},
 {'language': 'bem', 'name': 'Bemba'},
 {'language': 'bn', 'name': 'Bengali'},
 {'language': 'bew', 'name': 'Betawi'},
 {'language':

In [221]:
src_lan, tgt_lan

('en', 'es')

In [222]:
# sample test
sentence = sents[0]
test = translate_text(src_lan, tgt_lan, sentence)
test

{'translatedText': 'Este documento es un resumen del Informe Público Europeo de Evaluación (EPAR).',
 'input': 'This document is a summary of the European Public Assessment Report ( EPAR ) .'}

In [223]:
# translating all the sentences from English to target language
tgt_sents = []
for s in tqdm(sents):
    tgt_s = translate_text(src_lan, tgt_lan, s)
    tgt_sents.append(tgt_s)

100%|██████████| 121/121 [00:42<00:00,  2.85it/s]


In [224]:
tgt_sents = [s['translatedText'] for s in tgt_sents]

[{'translatedText': 'Este documento es un resumen del Informe Público Europeo de Evaluación (EPAR).', 'input': 'This document is a summary of the European Public Assessment Report ( EPAR ) .'}, {'translatedText': 'Explica cómo el Comité de Medicamentos de Uso Humano (CHMP) evaluó los estudios realizados para llegar a sus recomendaciones sobre cómo utilizar el medicamento.', 'input': 'It explains how the Committee for Medicinal Products for Human Use ( CHMP ) assessed the studies performed , to reach their recommendations on how to use the medicine .'}, {'translatedText': 'Si necesita más información sobre su enfermedad o su tratamiento, lea el prospecto (también incluido en el EPAR) o póngase en contacto con su médico o farmacéutico.', 'input': 'If you need more information about your medical condition or your treatment , read the Package Leaflet ( also part of the EPAR ) or contact your doctor or pharmacist .'}, {'translatedText': 'Si desea más información sobre las bases de las recom

In [225]:
# output
trans_df = pd.DataFrame({
    'ID': sids
    , 'English': sents
    , language+ ' Translation': tgt_sents
    })
trans_df.head()

,ID,English,Spanish Translation
0,d000.s000,This document is a summary of the European Pub...,Este documento es un resumen del Informe Públi...
1,d000.s001,It explains how the Committee for Medicinal Pr...,Explica cómo el Comité de Medicamentos de Uso ...
2,d000.s002,If you need more information about your medica...,Si necesita más información sobre su enfermeda...
3,d000.s003,If you want more information on the basis of t...,Si desea más información sobre las bases de la...
4,d000.s004,What is Alimta ?,¿Qué es Alimta?


In [226]:
# Save to TSV 
trans_df.to_csv(TRANS_FILE, sep='\t', index=False)

In [277]:
# load TSV 
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

,ID,English,Chinese Translation
0,d001.s001,This document is a summary of the European Pub...,本文件是欧洲公共评估报告 (EPAR) 的摘要。
1,d001.s002,It explains how the Committee for Medicinal Pr...,它解释了人用药品委员会 (CHMP) 如何评估所进行的研究，以得出关于如何使用药物的建议。
2,d001.s003,If you need more information about your medica...,如果您需要更多有关您的医疗状况或治疗的信息，请阅读药品说明书（也是 EPAR 的一部分）或联...
3,d001.s004,If you want more information on the basis of t...,如果您想了解有关 CHMP 建议的更多信息，请阅读科学讨论（也是 EPAR 的一部分）。
4,d001.s005,What is Alimta ?,什么是 Alimta？


In [278]:
trans_sents = trans_df[language+ " Translation"].tolist()

In [289]:
# combine the tokens in the sentence 
for x in range(len(trans_sents)):
    trans_sents[x] = " ".join(trans_sents[x])

[['本', '文件', '是', '欧洲', '公共', '评估', '报告', '(', 'EPAR', ')', '的', '摘要', '。'], ['它', '解释', '了', '人用', '药品', '委员会', '(', 'CHMP', ')', '如何', '评估所', '进行', '的', '研究', '，', '以', '得出', '关于', '如何', '使用', '药物', '的', '建议', '。'], ['如果', '您', '需要', '更', '多', '有关', '您', '的', '医疗', '状况', '或', '治疗', '的', '信息', '，', '请', '阅读', '药品', '说明书', '（', '也', '是', 'EPAR', '的', '一部分', '）', '或', '联系', '您', '的', '医生', '或', '药剂师', '。'], ['如果', '您', '想', '了解', '有关', 'CHMP', '建议', '的', '更', '多', '信息', '，', '请', '阅读', '科学', '讨论', '（', '也', '是', 'EPAR', '的', '一部分', '）', '。'], ['什么', '是', 'Alimta', '？'], ['Alimta', '是', '一种', '粉末', '，', '可', '制成', '输液', '溶液', '（', '滴入', '静脉', '）', '。'], ['它', '含有', '活性', '成分', '培美曲', '塞', '。'], ['Alimta', '的', '用途', '是', '什么', '？'], ['Alimta', '用于', '治疗', '两种', '类型', '的', '癌症', '：', '胸膜', '间皮瘤', '（', '肺部', '内膜', '癌症', '，', '通常', '由', '接触', '石棉', '引起', '）', '。'], ['当', '癌症', '无法', '切除', '（', '不能', '仅', '通过', '手术', '切除', '）', '且', '为', '恶性', '（', '已经', '扩散', '或', '可能', '轻易', '扩散', '到', '身体

# EVALUATE QUALITY
This can only be done for those languages in which you have gold translations. We have provided gold translations for Spanish, Italian, Chinese, Bengali, and Farsi.

In [251]:
gold_df = pd.read_csv(GOLD_FILE,sep="\t")
gold_sents = gold_df["Sentence"].tolist()

In [290]:
def calculate_bleu(reference_sentences, candidate_sentences):
    """
    Calculate the BLEU score between two lists of sentences.

    :param reference_sentences: List of reference sentences
    :param candidate_sentences: List of candidate sentences
    :return: BLEU score
    """
    references = [[ref.split()] for ref in reference_sentences]
    candidates = [cand.split() for cand in candidate_sentences]
    
    # Calculate sentence-level BLEU scores
    sentence_bleu_scores = [sentence_bleu( ref,cand, weights=(0.25,0.25,0.25,0.25)) for ref, cand in zip(references, candidates)]
    
    # Calculate corpus-level BLEU score
    corpus_bleu_score = corpus_bleu(references, candidates, weights=(0.25,0.25,0.25,0.25))
    
    return sentence_bleu_scores, corpus_bleu_score




In [291]:
sentence_scores, corpus_score = calculate_bleu(gold_sents,trans_sents)
print(f"Sentence-level BLEU scores: {sentence_scores}")
print(f"Corpus-level BLEU score: {corpus_score}")


['本 文件 是 欧洲 公共 评估 报告 ( EPAR ) 的 摘要 。 ', '它 解释 了 人 用 药品 委员会 ( CHMP ) 如何 评估 所 进行 的 研究 ， 以 得出 关于 如何 使用 药物 的 建议 。 ', '如果 您 需要 更 多 有关 您 的 医疗 状况 或 治疗 的 信息 ， 请 阅读 药品 说明书 （ 也 是 EPAR 的 一部分 ） 或 联系 您 的 医生 或 药剂师 。 ', '如果 您 想 了解 有关 CHMP 建议 的 更 多 信息 ， 请 阅读 科学 讨论 （ 也 是 EPAR 的 一部分 ） 。 ', '什么 是 Alimta ？ ', 'Alimta 是 一 种 粉末 ， 可 制成 输液 溶液 （ 滴入 静脉 ） 。 ', '它 含有 活性 成分 培美 曲塞 。 ', 'Alimta 的 用途 是 什么 ？ ', 'Alimta 用于 治疗 两 种 类型 的 癌症 ： 胸膜 间 皮 瘤 （ 肺部 内膜 癌症 ， 通常 由 接触 石棉 引起 ） 。 ', '当 癌症 无法切除 （ 不能 仅 通过 手术 切除 ） 且 为 恶性 （ 已经 扩散 或 可能 轻易 扩散 到 身体 其他 部位 ） ， 且 之前 未 接受 过 化疗 （ 抗癌 药物 ） 的 晚期 或 转移 性 非 小 细胞 肺癌 患者 （ 未 影响 鳞 状 细胞 ） 时 ， Alimta 与 顺 铂 （ 另 一 种 抗癌 药物 ） 一起 使用 。 ', '对于 以前 未 接受 过 治疗 的 患者 ， Alimta 与 顺铂 联合 使用 ； 对于 以前 接受 过 其他 类型 化疗 的 患者 ， Alimta 可 单独 使用 。 ', '这 种 药 必须 凭 处方 才 能 买到 。 ', 'Alimta 如何 使用 ？ ', 'Alimta 只 能 在 具有 抗癌 化疗 使用 资质 的 医生 的 监督 下 使用 。 ', 'Alimta 的 推荐 剂量 为 每 平方米 体表 面积 500 毫克 （ 根据 患者 的 身高 和 体重 计算 ） 。 ', '每 三 周 进行 一 次 静脉 输 注 ， 每次 持续 10 分钟 。 ', '为了 减少 副作用 ， 患者 在 使用 Alimta 治疗 期间 应该 服用 皮质类固醇 和 叶酸 （ 一 种 维生素 ） ， 并 注射 维生素 B1

/usr/local/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_